In [1]:
import csv 
import pandas as pd
import numpy as np

CSV File Paths

In [2]:
facilities_path = '/home/elena/Documents/affirm-take-home-interview-sept-2018/large/facilities.csv'
banks_path = '/home/elena/Documents/affirm-take-home-interview-sept-2018/large/banks.csv'
covenants_path = '/home/elena/Documents/affirm-take-home-interview-sept-2018/large/covenants.csv'
loans_path = '/home/elena/Documents/affirm-take-home-interview-sept-2018/large/loans.csv'

Using Pandas to read CSV files into Dataframes

In [3]:
DF_FACILITIES = pd.read_csv(facilities_path)
DF_BANKS = pd.read_csv(banks_path)
DF_COVENANTS = pd.read_csv(covenants_path)
DF_LOANS = pd.read_csv(loans_path)

In [35]:
DF_FACILITIES.head()

,amount,interest_rate,id,bank_id
0,1779071.0,0.04,5,1
1,23549.0,0.08,14,1
2,2127767.0,0.06,9,2
3,1991011.0,0.06,12,5
4,1938038.0,0.06,11,3


In [33]:
for attribute in DF_COVENANTS.columns:
    print('FEATURE:', attribute, ' UNIQUE VALUES:', len(DF_COVENANTS[attribute].unique()))

FEATURE: facility_id  UNIQUE VALUES: 15
FEATURE: max_default_likelihood  UNIQUE VALUES: 5
FEATURE: bank_id  UNIQUE VALUES: 5
FEATURE: banned_state  UNIQUE VALUES: 28


In [5]:
def clean_facility_data(DF_FACILITIES):
    DF_CLEAN_FACILITIES = DF_FACILITIES.rename(columns = {'id': 'facility_id'}).sort_values(by='facility_id').reset_index(drop=True)
    DF_CLEAN_FACILITIES.index = DF_CLEAN_FACILITIES.index + 1
    return DF_CLEAN_FACILITIES.to_dict(orient = 'index')

In [61]:
def get_covenant(COVENANT_DATA, ):
    for index, covenant in COVENANT_DATA.iterrows():
            if (loan['default_likelihood'] < covenant['max_default_likelihood']) & (loan['state'] != covenant['banned_state']):
                facility_status = FACILITY_DATA.get(covenant['facility_id'])
                difference = facility_status.get('amount') - loan['amount']
                if (difference) > 0:
                    facility_status['amount'] = difference
                    FACILITY_DATA[covenant['facility_id']] = facility_status
                    DF_ASSIGNMENTS = DF_ASSIGNMENTS.append({'loan_id': loan['id'], 'facility_id':covenant['facility_id']}, ignore_index=True)
                    break
    return(DF_ASSIGNMENTS)
get_covenant('COVENANT_DATA, )

In [64]:
def create_assignment_table(LOAN_DATA, COVENANT_DATA, FACILITY_DATA):
    COVENANT_DATA = COVENANT_DATA.fillna(1) #.sort_values(by=['facility_id'])
    DF_ASSIGNMENTS =  pd.DataFrame(columns=['loan_id', 'facility_id']) #initialize with NaN's same length as loan csv
    FACILITY_DATA = clean_facility_data(FACILITY_DATA)
    for index, loan in LOAN_DATA.iterrows():
        for index, covenant in COVENANT_DATA.iterrows():
            if (loan['default_likelihood'] < covenant['max_default_likelihood']) & (loan['state'] != covenant['banned_state']):
                facility_status = FACILITY_DATA.get(covenant['facility_id'])
                difference = facility_status.get('amount') - loan['amount']
                if (difference) > 0:
                    facility_status['amount'] = difference
                    FACILITY_DATA[covenant['facility_id']] = facility_status
                    DF_ASSIGNMENTS = DF_ASSIGNMENTS.append({'loan_id': loan['id'], 'facility_id':covenant['facility_id']}, ignore_index=True)
                    break
    return DF_ASSIGNMENTS

In [65]:
df_test = create_assignment_table(DF_LOANS, DF_COVENANTS, DF_FACILITIES)
df_test.to_csv(path_or_buf = 'assignments_test.csv', index=False)

In [66]:
def get_expected_yeild(default_likelihood, loan_interest_rate, amount, facility_interest_rate):
    return int((1 - default_likelihood) * loan_interest_rate * amount - (default_likelihood * amount) - (facility_interest_rate * amount))

In [67]:
def create_yeild_table(assigned_csv, LOAN_DATA, FACILITY_DATA):
    DF_ASSIGNED = pd.read_csv(assigned_csv)
    LOAN_DATA = LOAN_DATA.rename(columns = {'id':'loan_id', 'interest_rate':'loan_interest_rate'})
    FACILITY_DATA = FACILITY_DATA.rename(columns = {'id':'facility_id', 'amount':'facility_amount'})
    ASSIGNED_LOANS = pd.merge(DF_ASSIGNED, LOAN_DATA, on='loan_id')
    ALL_TOGETHER_NOW = pd.merge(FACILITY_DATA, ASSIGNED_LOANS, on = 'facility_id')
    DF_YIELD =  pd.DataFrame(columns=['facility_id', 'expected_yield'])
    for index, assignment in ALL_TOGETHER_NOW.iterrows():
        facility_amount, interest_rate, facility_id, bank_id,loan_id,loan_interest_rate, amount, default_likelihood, state = assignment
        expected_yield = get_expected_yeild(default_likelihood, loan_interest_rate, amount, interest_rate)
        DF_YIELD = DF_YIELD.append({'facility_id': int(facility_id), 'expected_yield':expected_yield}, ignore_index=True)
    return DF_YIELD

In [68]:
yield_test = create_yeild_table('assignments_test.csv', DF_LOANS, DF_FACILITIES)

In [69]:
yield_test = yield_test.groupby(['facility_id']).sum()

In [70]:
yield_test.to_csv(path_or_buf = 'yield_test.csv')